In [1]:
pa = dirname(@__DIR__)

pap = joinpath(pa, "Project.toml")

pas = joinpath(pa, "src")

pat = joinpath(pa, "test")

"/Users/kwat/craft/OnePiece.jl/test"

In [2]:
sr_ = [na for na in readdir(pas) if isdir(joinpath(pas, na))]

te_ = [splitext(na)[1] for na in readdir(pat) if endswith(na, ".ipynb")]

te_ = [te for te in te_ if !(te in ("runtests",))]

symdiff(sr_, te_)

1-element Vector{String}:
 "speed"

In [3]:
for (id, te) in enumerate(te_)
    
    println(id)
    
    nb = joinpath(pat, "$te.ipynb")

    if id < 1
        
        println("Skipped.")

        continue

    end

    println("Running ", nb, " (", id, ")")

    run(
        `jupyter-nbconvert --execute --ExecutePreprocessor.timeout=-1 --clear-output --inplace $nb`,
    )

end

1
Running /Users/kwat/craft/OnePiece.jl/test/constant.ipynb (1)


[NbConvertApp] Converting notebook /Users/kwat/craft/OnePiece.jl/test/constant.ipynb to notebook


Starting kernel event loops.


[NbConvertApp] Writing 5317 bytes to /Users/kwat/craft/OnePiece.jl/test/constant.ipynb


2
Running /Users/kwat/craft/OnePiece.jl/test/dataframe.ipynb (2)


...

[NbConvertApp] Writing 12203 bytes to /Users/kwat/craft/OnePiece.jl/test/tensor_function.ipynb


25
Running /Users/kwat/craft/OnePiece.jl/test/vector.ipynb (25)


[NbConvertApp] Converting notebook /Users/kwat/craft/OnePiece.jl/test/vector.ipynb to notebook


Starting kernel event loops.


[NbConvertApp] Writing 5384 bytes to /Users/kwat/craft/OnePiece.jl/test/vector.ipynb
